In [7]:
%matplotlib inline
import numpy as np
from glob import glob
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from venn import venn
from collections import Counter

In [61]:
### load data
v = 'combined_default_srv' 
dir_path = '/Users/osipova/Documents/LabDocs/NectarivoryProject/absrel/absrel_analysis_2024/{}/ClusterProfiler/'.format(v)

all_results = {}

nectar_clades    = ['hmmbrds', 'honeyeaters', 'nectar_parrots', 'sunbirds']
nonnectar_clades = ['swifts', 'falcons', 'lyrebirds', 'passerides']

clades = nectar_clades + nonnectar_clades
for clade in clades:
    file_name = 'goenrich.{}.under_selection_per_clade_0.05.tsv'.format(clade)
    file_data = glob(dir_path + file_name)[0]
    df = pd.read_csv(file_data, sep='\t')
    all_results[clade] = df

In [62]:
### Count in how many groups each GO is present

def count_gos(clades):
    all_goids = []
    for clade in clades:
        all_goids.extend(all_results[clade][all_results[clade]['pvalue'] < 0.05]['ID'])
    return Counter(all_goids)

nonnectar_go_count = count_gos(nonnectar_clades)
nectar_go_count = count_gos(nectar_clades)

rank3_nonnectar_go = [k for k in nonnectar_go_count if nonnectar_go_count[k] >= 3]
rank2_nectar_go = [k for k in nectar_go_count if nectar_go_count[k] >= 2]
rank3_nectar_go = [k for k in nectar_go_count if nectar_go_count[k] >= 3]
print(len(rank3_nonnectar_go))
print(len(rank2_nectar_go))
print(len(rank3_nectar_go))


12
92
14


In [58]:
### Exclude control GO ids from target

data_rank3_nectar = []
for clade in nectar_clades:
    df_clade = all_results[clade]
    filt_df_clade = df_clade[(df_clade['ID'].isin(rank3_nectar_go) 
                              & (~df_clade['ID'].isin(rank3_nonnectar_go)))][['ID', 'Description', 'pvalue']]
    data_rank3_nectar.append(filt_df_clade)
    
merged_df = pd.DataFrame({'ID': [], 'Description': []})
for df in data_rank3_nectar:
    merged_df = pd.merge(merged_df, df, on=['ID', 'Description'], how='outer')

/Users/osipova/local/Homebrew/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'pvalue_x'} in the result is deprecated and will raise a MergeError in a future version.
  if sys.path[0] == '':


In [59]:
merged_df

,ID,Description,pvalue_x,pvalue_y,pvalue_x,pvalue_y
0,GO:0043547,positive regulation of GTPase activity,0.000223,0.037759,NaN,0.045945
1,GO:0043087,regulation of GTPase activity,0.001627,0.032467,NaN,0.031793
2,GO:0030322,stabilization of membrane potential,0.003744,0.001585,0.017263,0.017263
3,GO:0097623,potassium ion export across plasma membrane,0.006020,0.031297,NaN,0.023558
4,GO:0071805,potassium ion transmembrane transport,0.008503,0.000983,NaN,0.004735
5,GO:0050772,positive regulation of axonogenesis,0.014540,0.004220,NaN,0.013141
6,GO:0006813,potassium ion transport,0.016759,0.000601,NaN,0.008722
7,GO:0048008,platelet-derived growth factor receptor signal...,0.023966,0.008745,NaN,0.032326
8,GO:0060457,negative regulation of digestive system process,0.040221,0.023023,0.017263,NaN
9,GO:0045780,positive regulation of bone resorption,0.044695,NaN,0.019269,0.019269
